# Noodlification and unification on an example

In [1]:
import awalipy
import noodler

[Warning] The python module awalipy relies on compilation executed "on-the-fly" depending on the context (type of weights, of labels, etc.). As a result, the very first call to a given function in a given context may take up to 10 seconds. 


## query description
The string equation problem with variables $\mathsf{Var}$ is given by two components:
 1. equation $e$ of the form $e_l = e_r$ where $e_l, e_r \in \mathsf{Var}^+$. In the code, we use `e_l` and `e_r`.
 2. dictionary `constraints`${}\colon \mathsf{Var} \to \mathsf{RE(\mathsf{Var})}$. The regular expressions must be given as strings parsable by Awali, see [this guide](http://files.vaucanson-project.org/1.0/gs/RationalExpression.html) for more details.

In the running example, we use variables `x`,`y`, and `z` with alphabet `a,b,c`. In the following, we create an equation $xyz = xyxy$ with regular constraints
* $L_x = L_z = \Sigma^+$ and 
* $L_y = \Sigma \Sigma^+.$

In [2]:
eq = noodler.StringEquation("zxy", "xyyx")

alph = "abc"
Sigma_exp = f"({'+'.join(alph)})"
Sigma_plus = f"{Sigma_exp}{Sigma_exp}*"
constraints = {
    "x" : f"{Sigma_exp}(a){Sigma_plus}",
    "y" : f"{Sigma_exp}{Sigma_plus}",
    "z" : Sigma_plus
}
for var in constraints:
    print(f"{var}: {constraints[var]}")

x: (a+b+c)(a)(a+b+c)(a+b+c)*
y: (a+b+c)(a+b+c)(a+b+c)*
z: (a+b+c)(a+b+c)*


In [3]:
aut_constraints = {var : awalipy.RatExp(c, alphabet=alph).exp_to_aut() for var, c in constraints.items()}

query = noodler.AutSingleSEQuery(eq, aut_constraints)
print(query.eq)
query.show_constraints()

zxy = xyyx


In [4]:
n = noodler.QueueNoodler(query)
print(query.eq)
n.one_solution()
print(f"Other {len(n.Q)} unprocessed noodles are in the queue")
n.solutions[0].show_constraints()

zxy = xyyx
Other 742 unprocessed noodles are in the queue


In [6]:
switched = query.switched()
n = noodler.QueueNoodler(switched)
print(switched.eq)
n.one_solution()
print(f"Other {len(n.Q)} unprocessed noodles are in the queue")
n.solutions[0].show_constraints()

xyyx = zxy
Other 9609 unprocessed noodles are in the queue


In [10]:
%%timeit
n = noodler.QueueNoodler(query)
n.one_solution()

207 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
n = noodler.QueueNoodler(switched)
n.one_solution()

9.54 ms ± 525 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
